In [ ]:
%autoreload 2

In [ ]:
import os

import anndata as ad
import torch

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.chdir("SCOOTR/run")
os.getcwd()

In [111]:
USE_ADT2GEX = True
if USE_ADT2GEX:
    #adt2gex
    test_path = "../datasets/phase2-private-data/match_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
else:
    #atac2gex
    test_path = "../datasets/phase2-private-data/match_modality/openproblems_bmmc_multiome_phase2_rna/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_"

prediction_path = "GEX2ADTH0.01.h5ad"

par = {
    "input_test_prediction": prediction_path,
    "input_test_sol": f"{test_path}test_sol.h5ad",
}

prediction_test = ad.read_h5ad(par["input_test_prediction"])
sol_test = ad.read_h5ad(par["input_test_sol"])

In [112]:
X = prediction_test.X.toarray()
X = torch.tensor(X)
X

Xsol = torch.tensor(sol_test.X.toarray())
Xsol.argmax(1)
# Order the columns of the prediction matrix so that the perfect prediction is the identity matrix
X = X[:, Xsol.argmax(1)]

In [ ]:
labels = torch.arange(X.shape[0])
forward_accuracy = (torch.argmax(X, dim=1) == labels).float().mean().item()
backward_accuracy = (
    (torch.argmax(X, dim=0) == labels).float().mean().item()
)
avg_accuracy = 0.5 * (forward_accuracy + backward_accuracy)
print(forward_accuracy, backward_accuracy, "top1-acc:", avg_accuracy)

In [ ]:
_, top_indexes_forward = X.topk(5, dim=1)
_, top_indexes_backward = X.topk(5, dim=0)
l_forward = labels.expand(5, X.shape[0]).T
l_backward = l_forward.T
top5_forward_accuracy = (
    torch.any(top_indexes_forward == l_forward, 1).float().mean().item()
)
top5_backward_accuracy = (
    torch.any(top_indexes_backward == l_backward, 0).float().mean().item()
)
top5_avg_accuracy = 0.5 * (top5_forward_accuracy + top5_backward_accuracy)

print(top5_forward_accuracy, top5_backward_accuracy, "top5-acc:", top5_avg_accuracy)

### For soft predictions, the competition score can be made equal to the forward accuracy (or backward accuracy) by putting 1 at the max of each row (or each column) and 0 elsewhere

In [ ]:
logits_row_sums = X.clip(min=0).sum(dim=1)
top1_competition_metric = X.clip(min=0).diag().div(logits_row_sums).mean().item()
top1_competition_metric